In [1]:
!pip --version

# To install pytorch -------------------------------------------------------------------------------------
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# To install Tensorflow ----------------------------------------------------------------------------------
# Requires the latest pip
!pip install --upgrade pip

# Current stable release for CPU and GPU
!pip install tensorflow

# To install Flax ----------------------------------------------------------------------------------------
!pip install flax

# or to install the latest version of Flax:
!pip install --upgrade git+https://github.com/google/flax.git

# Set up transformers
!pip install git+https://github.com/huggingface/transformers

# Check if installed
!python -c "from transformers import pipeline; print(pipeline('sentiment-analysis')('I love you'))"

!pip install transformers[torch]

!pip install accelerate -U

!pip install -q datasets peft evaluate

!pip install opacus

!pip install --upgrade peft

pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Cloning https://github.com/google/flax.git to /tmp/pip-req-build-3zo6q4y3
  Running command git clone --filter=blob:none --quiet https://github.com/google/flax.git /tmp/pip-req-build-3zo6q4y3
  Resolved https://github.com/google/flax.git to commit 85dfad242e56098849dbf05e7e4657b3a40820f9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for flax: filename=flax-0.8.0-py3-none-any.whl size=635291 sha256=8e9cf1867c346413a073bf0b199e706f1f02ca6b55da310db033762820b1da77
  Stored in directory: /tmp/p

In [13]:
from transformers import (AutoModelForSequenceClassification,AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer )
from peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    PeftType,
    PrefixTuningConfig,
    PromptEncoderConfig,
    PromptTuningConfig,
)
from peft import ( get_peft_config, get_peft_model, get_peft_model_state_dict, set_peft_model_state_dict, PeftModelForSeq2SeqLM, PeftType,PromptEncoderConfig,PeftModelForSequenceClassification)
from peft import PromptEmbedding, PromptTuningConfig
from datasets import load_dataset
import evaluate
from transformers import AdamW
import torch
import numpy as np
import pandas as pd
from peft import LoraModel, LoraConfig
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from datasets import Dataset
import opacus
from opacus import PrivacyEngine
from opacus.grad_sample import GradSampleModule
from opacus.validators import ModuleValidator


dataset = load_dataset("sst2")
model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny", num_labels=2) # as output 0 or 1
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny", padding_side = "right")
model = model.to('cuda')
# optimizer = AdamW(model.parameters(), lr= 0.005, eps = 1e-8)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
config = LoraConfig(
    task_type="TEXT_PAIR",
    r= 64,
    lora_alpha=32,
    lora_dropout=0.01,
    bias = "lora_only"
)

model = PeftModelForSequenceClassification(model, config)
# model = get_peft_model(model, config)
print(model.print_trainable_parameters())

# model = LoraModel(model, config, "default")

# model = get_peft_model(model, config)
# model.print_trainable_parameters()

model = ModuleValidator.fix(model)
model = GradSampleModule(model)

# l = []

optimizer = AdamW(model.parameters(), lr= 1e-4, eps = 1e-8)

In [ ]:
train = 0
total = 0
for p in model.base_model.model.bert.encoder.layer.parameters():
  total+=1
  if p.requires_grad:
    train+=1
print(train, total)

In [ ]:
model

In [16]:
# Preprocessing
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from datasets import Dataset

# # Assuming the dataset has a 'train' split, modify this according to your dataset's splits
# data_split = dataset['train']

# # Convert the dataset split to a pandas DataFrame for easier splitting
# df = data_split.to_pandas()

# # Split the dataset into train and test sets using train_test_split from sklearn
# # df1, df2 = train_test_split(df, test_size = 0.5, random_state = 42)
# train_df, test_df = train_test_split(df , test_size=0.2, random_state=42)

# # Convert the splits back to datasets
# train_dataset = train_df.reset_index(drop=True)
# test_dataset = test_df.reset_index(drop=True)

# train_dataset = Dataset.from_pandas(train_dataset)
# test_dataset = Dataset.from_pandas(test_dataset)

x_train = list(dataset["train"]["sentence"])
y_train = list(dataset["train"]["label"])

x_test = list(dataset["test"]["sentence"])
y_test = list(dataset["test"]["label"])

X_train_tokenized = tokenizer(x_train, padding=True, truncation=True, max_length = 256)
# X_val_tokenized = tokenizer(x_validation, padding=True, truncation=True, max_length=512)
X_test_tokenized = tokenizer(x_test, padding=True, truncation = True, max_length = 256)

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]).to('cuda') for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx]).to('cuda')
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
# val_dataset = Dataset(X_val_tokenized, y_validation)
test_dataset = Dataset(X_test_tokenized, y_test)

train_dataloader = DataLoader(train_dataset, batch_size = 1024, drop_last = True)
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size = 1024, drop_last = True)


In [ ]:

model.train()
privacy_engine = PrivacyEngine()
model, optimizer, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_dataloader,
    target_delta= 1 / len(train_dataloader),
    target_epsilon=  8,
    epochs= 3,
    max_grad_norm = 0.1,
    grad_sample_mode="hooks",
)



In [ ]:
!pip install tensorflow-privacy

In [ ]:
def accuracy(preds, labels):
    return (preds == labels).mean()

# define evaluation cycle
def evaluate(model):
    model.eval()
    loss_arr = []
    accuracy_arr = []

    for batch in test_dataloader:

        with torch.no_grad():
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss, logits = outputs[:2]

            log = logits.cpu()
            log = log.numpy()

            preds = np.argmax(log, axis=1)
            labels = batch['labels'].detach().cpu().numpy()

            loss_arr.append(loss.item())
            accuracy_arr.append(accuracy(preds, labels))

    model.train()
    return np.mean(loss_arr), np.mean(accuracy_arr)

model.train()

# Training loop
optimizer.zero_grad()  # Explicitly zero the gradient buffers

for epoch in range(60):  # Number of epochs
    model.train()
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()

        logits = outputs[1]
        predictions = torch.argmax(logits, dim = -1)

    eval_loss, eval_accuracy = evaluate(model)
    print("eval loss",eval_loss)
    print("accuracy: ",eval_accuracy)
    print("ends")

    model.train()  # Set the model back to training mode
